In [1]:
%bookmark HOME "/home/jupyter/meta_XLM/XLM" 
%cd -b HOME

(bookmark:HOME) -> /home/jupyter/meta_XLM/XLM
/home/jupyter/meta_XLM/XLM


In [12]:
%env csv_path=/home/jupyter
%env output_dir=/home/jupyter/data
%env data_type=para
%env languages=Bafia,Bulu
! python ../bible.py --csv_path $csv_path --output_dir $output_dir --data_type $data_type --languages $languages

env: csv_path=/home/jupyter
env: output_dir=/home/jupyter/data
env: data_type=para
env: languages=Bafia,Bulu
Bafia - Bulu
======= Read 7950 totals samples
======= Delete 4 samples
======= Save 7946 samples
Bulu
======= Read 31297 totals samples
======= Delete 2402 samples
======= Save 28895 samples


In [20]:
%env PARA=True          
%env MONO=True       
                   
%env PARA_PATH=/home/jupyter/data/Bafia-Bulu        
%env MONO_PATH=""  
%env SAME_VOCAB=True    
%env nCodes=10000             
%env shuf_n_samples=1000000   
%env threads_for_tokenizer=16 
%env test_size=10             
%env val_size=10              

# tools paths
%env TOKENIZE=tools/tokenizer_our.sh
%env LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
%env FASTBPE=tools/fastBPE/fast


%env OUTPATH=/home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed
# create output path
! mkdir -p $OUTPATH

! chmod +x $FASTBPE
! chmod +x ../build_meta_data_monoxlm.sh
! chmod +x tools/mosesdecoder/scripts/tokenizer/*.perl

%env n_samples=-1

%env sub_task=Bafia-Bulu:-1

env: PARA=True
env: MONO=True
env: PARA_PATH=/home/jupyter/data/Bafia-Bulu
env: MONO_PATH=""
env: SAME_VOCAB=True
env: nCodes=10000
env: shuf_n_samples=1000000
env: threads_for_tokenizer=16
env: test_size=10
env: val_size=10
env: TOKENIZE=tools/tokenizer_our.sh
env: LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
env: FASTBPE=tools/fastBPE/fast
env: OUTPATH=/home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed
env: n_samples=-1
env: sub_task=Bafia-Bulu:-1


In [16]:
%env add_para_data_to_mono_data=False
! ../build_meta_data_monoxlm.sh $sub_task $n_samples $add_para_data_to_mono_data

env: add_para_data_to_mono_data=False
dir /home/jupyter/models/africa/cluster1/data/Bafi/processed/fine_tune already exists
params ok !
*** Cleaning and tokenizing Bafia-Bulu data ... ***
Tokenizer Version 1.1
Language: Bafia
Number of threads: 8
*** Tokenized (+ lowercase + accent-removal) Bafia-Bulu.Bafia data to /home/jupyter/data/Bafia-Bulu/? ***
Tokenizer Version 1.1
Language: Bulu
Number of threads: 8
*** Tokenized (+ lowercase + accent-removal) Bafia-Bulu.Bulu data to /home/jupyter/data/Bafia-Bulu/? ***


*** split into train / valid / test ***
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error



***build the training set for BPE tokenization (10000 codes)***


***shuf ... Generating 1000000 random permutations of training data and store result in /home/jupyter/models/africa/cluster1/data/Bafi/processed/Bafia-Bulu/bpe.train***


***Lea

In [2]:
%env OUTPATH=/home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed
%env exp_id=maml
# If you don't have enough RAM or swap memory, leave these three parameters to True, otherwise you may get an error like this when evaluating 
# RuntimeError: copy_if failed to synchronize: cudaErrorAssert: device-side assert triggered
%env remove_long_sentences_train=True
%env remove_long_sentences_valid=True
%env remove_long_sentences_test=True
#--remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test

env: OUTPATH=/home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed
env: exp_id=maml
env: remove_long_sentences_train=True
env: remove_long_sentences_valid=True
env: remove_long_sentences_test=True


In [3]:
# le plus grand fichier à tensor_len phrases, on cherche le multiple de "batch_size" le plus proche de ce 
# nombre par valeur supérieur : epoch_size doit etre un multiple non nul de ce nombre (pour ne pas gaspiller) 

def getEpochSize(tensor_len, batch_size):
    i = tensor_len
    while True :
        if i%batch_size == 0 :
            return i//batch_size
        i = i + 1

import io

def n_lines(file_path):
    return len(io.open(file_path, encoding='UTF-8').read().split('\n'))

In [22]:
tensor_len = n_lines(file_path = "/home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed/Bafia-Bulu.Bafia.train")
print("tensor_len = " + str(tensor_len))

tensor_len = 6360


In [10]:
%env epoch_size=6360

%env lgs=Bafia-Bulu

# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

# stopping criterion (if criterion does not improve 10 times)
%env stopping_criterion=_valid_mlm_ppl,10
%env eval_bleu false

env: epoch_size=6360
env: lgs=Bafia-Bulu
env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: stopping_criterion=_valid_mlm_ppl,10
env: eval_bleu=false


In [ ]:
%env mlm_steps=Bafia,Bulu,Bafia-Bulu
%env batch_size=32
%env max_epoch=100
%env dump_path=/home/jupyter/models/africa/cluster1
! python train.py --exp_name mlm_tlm_BafiaBulu --exp_id $exp_id --dump_path $dump_path --data_path $OUTPATH --lgs $lgs --clm_steps '' --mlm_steps $mlm_steps --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --batch_size $batch_size --bptt 256 --optimizer adam,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --validation_metrics _valid_mlm_ppl --stopping_criterion $stopping_criterion --eval_bleu $eval_bleu --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

env: mlm_steps=Bafia,Bulu,Bafia-Bulu
env: batch_size=32
env: max_epoch=100
env: dump_path=/home/jupyter/models/africa/cluster1
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm-bis-vm
INFO - 05/24/20 04:19:58 - 0:00:00 - ============ Initialized logger ============
INFO - 05/24/20 04:19:58 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: []
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 32
                                     beam_size: 1
                           

INFO - 05/24/20 04:19:58 - 0:00:00 - 34569 words (8683 unique) in 794 sentences. 46 unknown words (38 unique) covering 0.13% of the data.
INFO - 05/24/20 04:19:58 - 0:00:00 - ========================== debug : 2

INFO - 05/24/20 04:19:58 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed/test.Bafia.pth ...
INFO - 05/24/20 04:19:58 - 0:00:00 - 35025 words (8683 unique) in 794 sentences. 52 unknown words (42 unique) covering 0.15% of the data.
INFO - 05/24/20 04:19:58 - 0:00:00 - ========================== debug : 3

INFO - 05/24/20 04:19:58 - 0:00:00 - ============ Monolingual data (Bulu)
INFO - 05/24/20 04:19:58 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed/train.Bulu.pth ...
INFO - 05/24/20 04:19:58 - 0:00:00 - 180730 words (8683 unique) in 6359 sentences. 0 unknown words (0 unique) covering 0.00% of the data.
INFO - 05/24/20 04:19:58 - 0:00:00 - ========================== debug : 4

INFO - 05/24/

INFO - 05/24/20 04:20:02 - 0:00:04 - Found 0 memories.
INFO - 05/24/20 04:20:02 - 0:00:04 - Found 6 FFN.
INFO - 05/24/20 04:20:02 - 0:00:04 - Found 102 parameters in model.
INFO - 05/24/20 04:20:02 - 0:00:04 - Optimizers: model
INFO - 05/24/20 04:20:02 - 0:00:05 - ============ Starting epoch 0 ... ============
INFO - 05/24/20 04:20:02 - 0:00:05 - Creating new training data iterator (pred,Bulu) ...
INFO - 05/24/20 04:20:05 - 0:00:07 - Creating new training data iterator (pred,Bafia,Bulu) ...
INFO - 05/24/20 04:20:07 - 0:00:09 - Creating new training data iterator (pred,Bafia) ...
INFO - 05/24/20 04:20:16 - 0:00:18 -       5 -   33.67 sent/s -   977.99 words/s - MLM-Bafia:  6.7095 || MLM-Bulu:  8.8718 || MLM-Bafia-Bulu:  6.9666 -  - model LR: 1.0000e-04
INFO - 05/24/20 04:20:30 - 0:00:32 -      10 -   35.81 sent/s -  2084.28 words/s - MLM-Bafia:  5.6604 || MLM-Bulu:  6.3187 || MLM-Bafia-Bulu:  5.7192 -  - model LR: 1.0000e-04
INFO - 05/24/20 04:20:43 - 0:00:45 -      15 -   34.89 sent/s 

**Train a (unsupervised/supervised) MT from a pretrained meta-model**

In [5]:
%env OUTPATH=/home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed
%env exp_id=maml
%env epoch_size=6360
%env lgs=Bafia-Bulu

env: OUTPATH=/home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed
env: exp_id=maml
env: epoch_size=6360
env: lgs=Bafia-Bulu


In [7]:
# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

%env eval_bleu=true
! chmod +x src/evaluation/multi-bleu.perl

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: eval_bleu=true


In [8]:
%env stopping_criterion=valid_mt_bleu,10
%env validation_metrics=valid_mt_bleu
#%env reload_model=dumped/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth,dumped/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth
%env reload_model=/home/jupyter/models/africa/cluster1/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth,/home/jupyter/models/africa/cluster1/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth

%env ae_steps=Bafia,Bulu
%env bt_steps=Bafia-Bulu-Bafia,Bulu-Bafia-Bulu

env: stopping_criterion=valid_mt_bleu,10
env: validation_metrics=valid_mt_bleu
env: reload_model=/home/jupyter/models/africa/cluster1/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth,/home/jupyter/models/africa/cluster1/mlm_tlm_BafiaBulu/maml/best-valid_mlm_ppl.pth
env: ae_steps=Bafia,Bulu
env: bt_steps=Bafia-Bulu-Bafia,Bulu-Bafia-Bulu


In [4]:
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1

%env max_epoch=100

# unsupervised MT
#! python train.py --exp_name UnSupMT_BafiaBulu --exp_id $exp_id --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: max_epoch=100


In [ ]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_BafiaBulu  /content/drive/"My Drive"/African_Translator/models/africa/cluster1

In [9]:
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1

%env max_epoch=100
%env batch_size=32
%env dump_path=/home/jupyter/models/africa/cluster1
# supervised MT
%env mt_steps=Bafia-Bulu,Bulu-Bafia          
! python train.py --exp_name SupMT_BafiaBulu --exp_id $exp_id  --dump_path $dump_path --reload_model $reload_model --data_path $OUTPATH --lgs $lgs --ae_steps $ae_steps --mt_steps $mt_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test    

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1
env: max_epoch=100
env: batch_size=32
env: dump_path=/home/jupyter/models/africa/cluster1
env: mt_steps=Bafia-Bulu,Bulu-Bafia
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm-bis-vm
INFO - 05/24/20 04:04:31 - 0:00:00 - ============ Initialized logger ============
INFO - 05/24/20 04:04:31 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: ['Bafia', 'Bulu']
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_s

INFO - 05/24/20 04:04:31 - 0:00:00 - 274456 words (8683 unique) in 6359 sentences. 0 unknown words (0 unique) covering 0.00% of the data.
INFO - 05/24/20 04:04:31 - 0:00:00 - ========================== debug : 1
INFO - 05/24/20 04:04:31 - 0:00:00 - ========================== debug : 2
INFO - 05/24/20 04:04:31 - 0:00:00 - Removed 0 empty sentences.
INFO - 05/24/20 04:04:31 - 0:00:00 - Removed 36 too long sentences.

INFO - 05/24/20 04:04:31 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluster1/data/Bafia_Bulu/processed/valid.Bafia.pth ...
INFO - 05/24/20 04:04:31 - 0:00:00 - 34569 words (8683 unique) in 794 sentences. 46 unknown words (38 unique) covering 0.13% of the data.
INFO - 05/24/20 04:04:31 - 0:00:00 - ========================== debug : 3
INFO - 05/24/20 04:04:31 - 0:00:00 - ========================== debug : 4
INFO - 05/24/20 04:04:31 - 0:00:00 - Removed 3 too long sentences.

INFO - 05/24/20 04:04:31 - 0:00:00 - Loading data from /home/jupyter/models/africa/cluste

In [ ]:
#! rsync -av --progress /content/meta_XLM/XLM/dumped/mlm_tlm_BafiaBulu  /content/drive/"My Drive"/African_Translator/models/africa/cluster1

In [ ]:
d=[]
while(1):
  d.append(d)